# Create Database and Schema on postgresql DB

createDBSQL = """CREATE DATABASE "AIQ"
    WITH 
    OWNER = postgres
    ENCODING = 'UTF8'
    CONNECTION LIMIT = -1;"""

createSchemaSQL = """ CREATE SCHEMA IF NOT EXISTS "AIQsales" AUTHORIZATION postgres; """


# install below module on the python environment.

! pip install requests== 2.31.0
! pip install pandas== 2.0.3
! pip install psycopg2== 2.9.9 
! pip installsqlalchemy== 1.4.39
! pip install aiohttp== 3.8.5



# Once Modules are installed and Database have been create, we can exceute this note book to create required tables on the postgresql server.

In [ ]:
import pandas as pd
from datetime import datetime
from pprint import pprint
import psycopg2

from AIQSales.Gettoken import AIQ_GetAuth 
from AIQSales.GetData import AIQ_GetData
from AIQSales.LoadToDB import To_postgres

from AIQSales.Setupdb import postgresConn

In [53]:
pg_local= {
    "host"      : "localhost",
    "database"  : "AIQ",
    "user"      : "postgres",
    "password"  : "root"
}

pgconn = postgresConn(pg_local)

Connecting to the PostgreSQL database...
Connection successful


In [56]:
cursor = pgconn.Getcursor()
conn =  pgconn.GetConnection()

In [ ]:

dropCustomerSQL =  "drop table IF EXISTS Dim_Customers ;"

createCustomerSQL = """CREATE TABLE IF NOT EXISTS Dim_Customers(
    id integer ,
    name character varying(30),
    username character varying(30),
    email character varying(30),
    phone character varying(50) NULL,
    website character varying(50) NULL,
    street character varying(100) NULL,
    city character varying(30),
    zipcode character varying(30),
    suite character varying(30),
    lat character varying(30) NULL,
    lng character varying(30) NULL,
    companyname character varying(50) NULL,
    catchPhrase character varying(100) NULL,
    bs character varying(100) NULL,
    inserted_date  date NOT NULL,
    inserted_by character varying(10) NULL,
    updated_date  date NOT NULL,
    updated_by character varying(10) NULL
    ) ;"""

dropCustomerStgSQL= "drop table IF  EXISTS Dim_Customers_stg ;"

createCustomerStgSQL =  """ CREATE TABLE IF NOT EXISTS Dim_Customers_stg(
    id integer ,
    name character varying(30),
    username character varying(30),
    email character varying(30),
    phone character varying(50) NULL,
    website character varying(50) NULL,
    street character varying(100) NULL,
    city character varying(30),
    zipcode character varying(30),
    suite character varying(30),
    lat character varying(30) NULL,
    lng character varying(30) NULL,
    companyname character varying(50) NULL,
    catchPhrase character varying(100) NULL,
    bs character varying(100) NULL,
    inserted_date  date NOT NULL,
    inserted_by character varying(10) NULL,
    updated_date  date NOT NULL,
    updated_by character varying(10) NULL
    ) ;"""


dropsalesSQL = "drop table IF  EXISTS sales;"

dropsalesStgSQL= "drop table IF  EXISTS  sales;"

createSalesSQL = """ CREATE TABLE IF NOT EXISTS  sales(
    order_id integer not NULL,
    customer_id integer not NULL,
    product_id integer not NULL,
    quantity integer not NULL,
    price DECIMAL(15,2) not NULL,
    order_date date NOT NULL,
    id integer not NULL,
    lat character varying(30) NULL, 
    lng character varying(30) NULL,
    weather character varying(30) NULL,
    description character varying(30) NULL,
    temp DECIMAL(15,2) not NULL,
    pressure DECIMAL(15,2) not NULL,
    humidity DECIMAL(15,2) not NULL, 
    grnd_level DECIMAL(15,2) not NULL,
    sea_level DECIMAL(15,2) not NULL,
    visibility character varying(30) NULL,
    wind_speed DECIMAL(15,2) not NULL,
    wind_deg DECIMAL(15,2) not NULL,
    wind_gust DECIMAL(15,2) not NULL,
    inserted_date date NOT NULL, 
    inserted_by character varying(10) NULL,
    updated_date date NOT NULL, 
    updated_by character varying(10) NULL
) ;"""


createSalesStgSQL = """ CREATE TABLE IF NOT EXISTS sales_stg(
    order_id integer not NULL,
    customer_id integer not NULL,
    product_id integer not NULL,
    quantity integer not NULL,
    price DECIMAL(15,2) not NULL,
    order_date date NOT NULL,
    id integer not NULL,
    lat character varying(30) NULL, 
    lng character varying(30) NULL,
    weather character varying(30) NULL,
    description character varying(30) NULL,
    temp DECIMAL(15,2) not NULL,
    pressure DECIMAL(15,2) not NULL,
    humidity DECIMAL(15,2) not NULL, 
    grnd_level DECIMAL(15,2) not NULL,
    sea_level DECIMAL(15,2) not NULL,
    visibility character varying(30) NULL,
    wind_speed DECIMAL(15,2) not NULL,
    wind_deg DECIMAL(15,2) not NULL,
    wind_gust DECIMAL(15,2) not NULL,
    inserted_date date NOT NULL, 
    inserted_by character varying(10) NULL,
    updated_date date NOT NULL, 
    updated_by character varying(10) NULL
) ; """



CreateSalePrimaryKey = """ALTER TABLE sales ADD CONSTRAINT pk_sales
  PRIMARY KEY (order_id); """


CreateCustomerPrimaryKey = """ ALTER TABLE Dim_Customers ADD CONSTRAINT pk_Dim_Customers
  PRIMARY KEY (id); """


CreateSalesForeignKey= """ ALTER TABLE sales ADD CONSTRAINT fk_sales
  FOREIGN KEY (customer_id) REFERENCES Dim_Customers(id); """


In [58]:
CreateQuery =  [ createSalesStgSQL, createSalesSQL, createCustomerStgSQL, createCustomerSQL,   CreateCustomerPrimaryKey]
DropQuery= [dropsalesSQL, dropsalesStgSQL, dropCustomerStgSQL, dropCustomerSQL ]

for query in  DropQuery:
    try:
        cursor.execute(query)
        conn.commit()
        print("SqlQuery ( {0} ) on the Database have been completed".format(query))

    except (Exception, psycopg2.DatabaseError) as error:
        #os.remove(tmp_df)
        message = "Error: %s" % error
        print(message)
        conn.rollback()
        cursor.close()
    

for query in  CreateQuery:
    try:
        cursor.execute(query)
        conn.commit()
        print("SqlQuery ( {0} ) on the Database have been completed".format(query))

    except (Exception, psycopg2.DatabaseError) as error:
        #os.remove(tmp_df)
        message = "Error: %s" % error
        print(message)
        conn.rollback()
        cursor.close()



SqlQuery ( drop table IF  EXISTS sales; ) on the Database have been completed
SqlQuery ( drop table IF  EXISTS  sales; ) on the Database have been completed
SqlQuery ( drop table IF  EXISTS Dim_Customers_stg ; ) on the Database have been completed
SqlQuery ( drop table IF EXISTS Dim_Customers ; ) on the Database have been completed
SqlQuery (  CREATE TABLE IF NOT EXISTS sales_stg(
    order_id integer not NULL,
    customer_id integer not NULL,
    product_id integer not NULL,
    quantity integer not NULL,
    price DECIMAL(15,2) not NULL,
    order_date date NOT NULL,
    id integer not NULL,
    lat character varying(30) NULL, 
    lng character varying(30) NULL,
    weather character varying(30) NULL,
    description character varying(30) NULL,
    temp DECIMAL(15,2) not NULL,
    pressure DECIMAL(15,2) not NULL,
    humidity DECIMAL(15,2) not NULL, 
    grnd_level DECIMAL(15,2) not NULL,
    sea_level DECIMAL(15,2) not NULL,
    visibility character varying(30) NULL,
    wind_spee

In [59]:
# check if the table have been created successfully or not.

cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)



('sales_stg',)
('sales',)
('dim_customers_stg',)
('dim_customers',)


In [60]:
# close the cursor

cursor.close()